## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-11-13-00-59 +0000,nypost,Outgoing BBC boss ‘very proud’ of its journali...,https://nypost.com/2025/11/11/world-news/bbcs-...
1,2025-11-11-12-58-50 +0000,nyt,Trump Pardons Runner Who Took Prohibited Short...,https://www.nytimes.com/2025/11/11/world/ameri...
2,2025-11-11-12-57-16 +0000,bbc,Anger over 'wanted notice' for Nigeria's ex-oi...,https://www.bbc.com/news/articles/cx2l48vkyvlo...
3,2025-11-11-12-56-16 +0000,bbc,Court rules asylum seekers can stay at Epping ...,https://www.bbc.com/news/articles/c8x1wz8jz0yo...
4,2025-11-11-12-51-03 +0000,bbc,UK unemployment rate rises to 5% as jobs marke...,https://www.bbc.com/news/articles/cdxrp7znkdlo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,43
156,shutdown,31
64,government,22
74,new,19
1,bbc,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
154,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...,137
140,2025-11-11-02-00-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...,128
225,2025-11-10-21-11-40 +0000,nypost,Trump pledges to ‘abide’ by deal to end govern...,https://nypost.com/2025/11/10/us-news/trump-pl...,122
124,2025-11-11-03-12-00 +0000,wsj,The decision by eight members of the Senate De...,https://www.wsj.com/politics/policy/democrats-...,120
127,2025-11-11-02-55-00 +0000,wsj,The Republican-led Senate late Monday passed a...,https://www.wsj.com/politics/policy/senate-pas...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
154,137,2025-11-11-01-13-00 +0000,wsj,Trump threatened to dock the pay of air-traffi...,https://www.wsj.com/politics/policy/trump-thre...
140,75,2025-11-11-02-00-00 +0000,wsj,Senate Minority Leader Chuck Schumer wasn’t on...,https://www.wsj.com/politics/policy/progressiv...
328,65,2025-11-10-14-58-51 +0000,nypost,Supreme Court rejects Kim Davis bid to overtur...,https://nypost.com/2025/11/10/us-news/supreme-...
327,59,2025-11-10-15-03-47 +0000,cbc,8 people dead in car explosion near Red Fort i...,https://www.cbc.ca/news/world/india-explosion-...
275,51,2025-11-10-17-49-00 +0000,wsj,The BBC reported that President Trump has thre...,https://www.wsj.com/politics/policy/bbc-chairm...
306,43,2025-11-10-16-35-00 +0000,wsj,Sen. Bernie Sanders and a group of Democratic ...,https://www.wsj.com/politics/policy/the-politi...
77,36,2025-11-11-09-24-46 +0000,nypost,Families sue Camp Mystic over deadly Texas flo...,https://nypost.com/2025/11/11/us-news/families...
318,33,2025-11-10-15-58-48 +0000,nypost,NYC realtor fired over controversial Zohran Ma...,https://nypost.com/2025/11/10/business/nyc-rea...
18,33,2025-11-11-12-00-00 +0000,wsj,The shutdown’s effect will probably be mostly ...,https://www.wsj.com/economy/government-shutdow...
311,32,2025-11-10-16-15-03 +0000,nypost,Saudi doctor who killed 6 after plowing car in...,https://nypost.com/2025/11/10/world-news/docto...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
